# 1. Pre-processing workflow

In [6]:
#Import libraries
import os
from calibrations import  build_calibration, calibrate_color_and_distortion, calibrate_color, calibrate_distortion

from aux_functions import obtain_pixel_metric, ungroup_pic
from model_class import ModelSegmentation
import pandas as pd

#Set paths of the files
working_directory="C:/Users/Pheno/Documents/database_almondcv2/"
chessboards=os.path.join(working_directory, "calibracion/chessboards") #folder with chessboard pitcures
raw_folder=os.path.join(working_directory,"pruebas_jorge")#folder with the pictures to calibrate
mtx_input_path=os.path.join(chessboards,"calibration_mtx.npz") #for distortion in npz format
standard_matrix_color=os.path.join(working_directory, "pruebas_jorge/28_10_CG-009.JPG") #picture of reference
output_calibrated=os.path.join(working_directory,"pruebas_jorge") #output folder for calibrated pictures

coin_model_path=os.path.join(working_directory,"")
info_table=os.path.join(working_directory,)

group_model_path=os.path.join(working_directory, "models/rectangle_2022_yolov11s_1280.pt")

## Color and Distortion calibration

In [3]:
# First build your distortion model based in chessboards
build_calibration(chessboardSize=(6, 8), frameSize=(5472,3648), dir_path=chessboards, 
                  image_format=".jpg", size_of_chessboard_squares_mm=20)

In [7]:
#Function for calibrate color and distortion
calibrate_color_and_distortion(raw_folder=raw_folder,mtx_input_path=mtx_input_path,output_calibrated=output_calibrated,
                                radius_param=10, standard_matrix=standard_matrix_color) #Standard matrix is a picture of reference to use instead of the original picture for error cases or simplicity

In [2]:
#Function for calibrate distortion only

calibrate_distortion(input_folder=raw_folder, mtx_input=mtx_input_path, output_path=output_calibrated, input_picture=None)

In [3]:
#Function for calibrate color only

calibrate_color(input_folder=raw_folder, output_path=output_calibrated,standard_matrix=standard_matrix_color,
                 force_standard_matrix="No")  #force_standard_matrix option uses in all the pictures the reference picture. In negative case, use only standard_matrix in error cases.

## Obtain pixel size

In [ ]:
#Deploy the model 

reference_model=ModelSegmentation(working_directory=working_directory)
contours_coin=reference_model.slice_predict_reconstruct(input_folder=output_calibrated, imgsz=640,
                                                         model_path=coin_model_path, slice_height=640, slice_width=640,
                                                         overlap_height_ratio=0.2, overlap_width_ratio=0.2,
                                                           retina_mask=True)

In [ ]:
#Load info table
info_data_df=pd.read_csv(info_table,sep="\t")
# If we use a calibrated dataset but the info table was previous we can include CL_ automatically with this line
info_data_df['Name_picture'] = info_data_df['Name_picture'].apply(lambda x: 'CL_' + x)

info_data_completed=obtain_pixel_metric(info_data=info_data_df, contours=contours_coin,
                                         output_directory=working_directory, reference=24.25) #reference in mm

## Ungroup pictures

In [ ]:
#Load group model
group_model=ModelSegmentation(working_directory=working_directory)
contours_groups=group_model.predict_model(model_path=group_model_path,
                               folder_input=output_calibrated,
                               imgsz=1280, check_result=False, max_det=2, retina_mask=False) #Retina mask not recommended here.

In [ ]:
#Obtain sample pictures and update info table. 
info_data_completed_path=os.path.join(working_directory, "info_data_completed_2022.txt")
info_data_completed=pd.read_csv(info_data_completed_path,sep="\t")

ungroup_pic(input_contours=contours_groups, output_path=working_directory, info_file=info_data_completed, axis="X") #axis indicate if the samples should be order according to Y or X axis